# Module 11 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Reinforcement Learning with Value Iteration

These are the same maps from Module 1 but the "physics" of the world have changed. In Module 1, the world was deterministic. When the agent moved "south", it went "south". When it moved "east", it went "east". Now, the agent only succeeds in going where it wants to go *sometimes*. There is a probability distribution over the possible states so that when the agent moves "south", there is a small probability that it will go "east", "north", or "west" instead and have to move from there.

There are a variety of ways to handle this problem. For example, if using A\* search, if the agent finds itself off the solution, you can simply calculate a new solution from where the agent ended up. Although this sounds like a really bad idea, it has actually been shown to work really well in video games that use formal planning algorithms (which we will cover later). When these algorithms were first designed, this was unthinkable. Thank you, Moore's Law!

Another approach is to use Reinforcement Learning which covers problems where there is some kind of general uncertainty in the actions. We're going to model that uncertainty a bit unrealistically here but it'll show you how the algorithm works.

As far as RL is concerned, there are a variety of options there: model-based and model-free, Value Iteration, Q-Learning and SARSA. You are going to use Value Iteration.

## The World Representation

As before, we're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
^       hills      5
~       swamp      7
x       mountains  impassible
```

When you go from a plains node to a forest node it costs 3. When you go from a forest node to a plains node, it costs 1. You can think of the grid as a big graph. Each grid cell (terrain symbol) is a node and there are edges to the north, south, east and west (except at the edges).

There are quite a few differences between A\* Search and Reinforcement Learning but one of the most salient is that A\* Search returns a plan of N steps that gets us from A to Z, for example, A->C->E->G.... Reinforcement Learning, on the other hand, returns  a *policy* that tells us the best thing to do in **every state.**

For example, the policy might say that the best thing to do in A is go to C. However, we might find ourselves in D instead. But the policy covers this possibility, it might say, D->E. Trying this action might land us in C and the policy will say, C->E, etc. At least with offline learning, everything will be learned in advance (in online learning, you can only learn by doing and so you may act according to a known but suboptimal policy).

Nevertheless, if you were asked for a "best case" plan from (0, 0) to (n-1, n-1), you could (and will) be able to read it off the policy because there is a best action for every state. You will be asked to provide this in your assignment.

We have the same costs as before. Note that we've negated them this time because RL requires negative costs and positive rewards:

In [1]:
costs = { '.': -1, '*': -3, '^': -5, '~': -7}
costs

{'.': -1, '*': -3, '^': -5, '~': -7}

and a list of offsets for `cardinal_moves`. You'll need to work this into your **actions**, A, parameter.

In [2]:
cardinal_moves = [(0,-1), (1,0), (0,1), (-1,0)]

For Value Iteration, we require knowledge of the *transition* function, as a probability distribution.

The transition function, T, for this problem is 0.70 for the desired direction, and 0.10 each for the other possible directions. That is, if the agent selects "north" then 70% of the time, it will go "north" but 10% of the time it will go "east", 10% of the time it will go "west", and 10% of the time it will go "south". If agent is at the edge of the map, it simply bounces back to the current state.

You need to implement `value_iteration()` with the following parameters:

+ world: a `List` of `List`s of terrain (this is S from S, A, T, gamma, R)
+ costs: a `Dict` of costs by terrain (this is part of R)
+ goal: A `Tuple` of (x, y) stating the goal state.
+ reward: The reward for achieving the goal state.
+ actions: a `List` of possible actions, A, as offsets.
+ gamma: the discount rate

you will return a policy: 

`{(x1, y1): action1, (x2, y2): action2, ...}`

Remember...a policy is what to do in any state for all the states. Notice how this is different than A\* search which only returns actions to take from the start to the goal. This also explains why reinforcement learning doesn't take a `start` state.

You should also define a function `pretty_print_policy( cols, rows, policy)` that takes a policy and prints it out as a grid using "^" for up, "<" for left, "v" for down and ">" for right. Use "x" for any mountain or other impassable square. Note that it doesn't need the `world` because the policy has a move for every state. However, you do need to know how big the grid is so you can pull the values out of the `Dict` that is returned.

```
vvvvvvv
vvvvvvv
vvvvvvv
>>>>>>v
^^^>>>v
^^^>>>v
^^^>>>G
```

(Note that that policy is completely made up and only illustrative of the desired output). Please print it out exactly as requested: **NO EXTRA SPACES OR LINES**.

* If everything is otherwise the same, do you think that the path from (0,0) to the goal would be the same for both A\* Search and Q-Learning?
* What do you think if you have a map that looks like:

```
><>>^
>>>>v
>>>>v
>>>>v
>>>>G
```

has this converged? Is this a "correct" policy? What are the problems with this policy as it is?


In [3]:
def read_world(filename):
    result = []
    with open(filename) as f:
        for line in f.readlines():
            if len(line) > 0:
                result.append(list(line.strip()))
    return result

---

In [4]:
small_world = read_world('small.txt')

In [5]:
small_world

[['.', '.', '.', '.', '.', '.'],
 ['.', '*', '*', '*', '*', '.'],
 ['.', '*', '*', '*', '*', '.'],
 ['.', '*', '*', 'x', '*', '.'],
 ['.', '*', '*', '*', '*', '.'],
 ['.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.']]

### Necessary Components for the Value Iteration Procedure
#### `reward(world:list, state:tuple, costs:dict)`
`reward(world:list, state:tuple, costs:dict)` provides the $R$ element in the Value Iteration procedure. It simply provides the reward of a `state` given the `costs` and the `world` provided. In the case that the state is out of bounds or the state is impassible (mountains), we pass 10 times the value of the worst reward (considering they are negative). 

In [6]:
def get_reward(world:list, state:tuple, costs:dict) -> int:
    max_width = len(world[0])
    max_height = len(world)
    if state[0] not in range(max_width) or state[1] not in range(max_height) or world[state[1]][state[0]] == 'x':
        return min(costs.values()) * 10
    else:
        return costs[world[state[1]][state[0]]]

In [7]:
# at the beginning, 
assert get_reward(small_world, (0,0), costs) == -1
# The following yields * which should have a reward of -3
# edge case when out of bounds at state (3,10), in this case we give the worst reward
assert get_reward(small_world, (3,10), costs) == -70
# case when we meet a mountain 'x', it should yield the worst case of -7*10
assert get_reward(small_world, (3,3), costs) == -70

#### transition(state:tuple, action:tuple, actions:tuple, desired_prob:float) -> list
The function `transition()` corresponds to the implementation of $T$ above in the problem statement. It will return a list of all the available `actions`. The desired `action` will receive the desired probability (`desired_prob`) while the other actions will receive an equal division of the residual probability (1-`desired_prob` / number of actions), once we have the probabilities and actions, we will implement them to the `state`. Notice that whenever the action results out of bounds, then we do not change the position of the agent. Instead we leave the agent where it originally started. This corresponds to my implementation of the 'bounce back' direction from the problem statement.

In [8]:
def transition(state:tuple, action:tuple, actions:tuple, desired_prob:float, world:list) -> list:
    unplanned_actions = [unplanned for unplanned in actions if unplanned != action]
    undesired_prob = (1-desired_prob) / len(unplanned_actions)
    t = [(desired_prob, action)]
    unplanned_actions = [(undesired_prob, undesired) for undesired in unplanned_actions]
    t += unplanned_actions
    imp_t = []
    for t_i in t:
        new_position = (t_i[1][0]+state[0],t_i[1][1]+state[1])
        if new_position[0] in range(len(world[0])) and new_position[1] in range(len(world)):
            imp_t.append((t_i[0],new_position))
        else:
            imp_t.append((t_i[0],state))
    return imp_t

In [9]:
# from the initial point (0,0) if the desired action is to go down, then (0,1) should receive 0.7 probability and the rest
# about 0.1
assert transition((0,0), (0,1), cardinal_moves, 0.7, small_world) == [(0.7, (0, 1)),
                                                                      (0.10000000000000002, (0, 0)),
                                                                      (0.10000000000000002, (1, 0)),
                                                                      (0.10000000000000002, (0, 0))]
# similarly, starting at (5,5) with a desired action of going left one (-1,0) then (4,5) should receive 0.7 probability
# while the others receive ~0.1
assert transition((5,5), (-1,0), cardinal_moves, 0.7, small_world) == [(0.7, (4, 5)),
                                                                       (0.10000000000000002, (5, 4)),
                                                                       (0.10000000000000002, (5, 5)),
                                                                       (0.10000000000000002, (5, 6))]

#### Helper Functions
##### `arg_max(sequence, function, world)`
`arg_max()` loops through the given values of a `function` and stores the best_score of such `function` along with the argument or variable that generates such best score. This function is used below when trying to find the max argument of a state in a given policy
##### `expected_utility(a, actions, state, U, desired_prob, world)`
`expected_utility()` computes the utility of a given transition (as computed in `transition()`) in a particular state. This function is used to find the best policy.
##### `best_policy(states, U, actions, desired_prob, world)`
`best_policy()` uses `arg_max()` and `expected_utility()` to find the action within `actions` which provides the highest policy value or utility in a given state given the policy `U`.

In [10]:
def best_policy(states, U, actions, desired_prob, world):
    pi = {}
    for state in states:
        pi[state] = arg_max(actions, lambda a: expected_utility(a, actions, state, U, desired_prob, world), world)
    return pi

In [11]:
def arg_max(sequence, function, world):
    best = sequence[0]
    best_score = function(best)
    for i in sequence:
        if i[0] in range(len(world[0])) and i[1] in range(len(world)):
            i_score = function(i)
            if i_score > best_score:            
                best, best_score = i, i_score                        
    return best

In [12]:
def expected_utility(a, actions, state, U, desired_prob, world):
    return sum([p * U[s1] for (p, s1) in transition(state, a, actions, desired_prob, world)])

### `value_iteration(world, costs, goal, reward, actions, gamma, desired_prob, epsilon)`
`value_iteration()` implements the pseudo-code provided in the book in Figure 17.6 (page 573). Some of the differences with the pseudo-code are related to the instructions provided in the problem statement and implemented in `reward()` and in `transition()`. 

In [13]:
def value_iteration(world, costs, goal, reward, actions, gamma, desired_prob, epsilon):
    width= len(world[0])
    height= len(world)
    states = [(x,y) for x in range(width) for y in range(height)]
    U_1 = dict([(s,0) for s in states])
    while True:
        U = U_1.copy()
        
        delta = 0
        for state in states:
            if state == goal:
                R = reward
            else:
                R = get_reward(world, state, costs)
            U_1[state] = R + gamma * max([sum([p * U[s1] for (p, s1) in transition(state, a, actions, desired_prob, world)
                                               if s1 in U]) 
                                          for a in actions])
            delta = max(delta, abs(U_1[state] - U[state]))

        if delta < epsilon:
            break
    return best_policy(states, U, actions, desired_prob, world)

In [14]:
pi = value_iteration(small_world, costs, (5,6), 10, cardinal_moves, 0.9, 0.7, 0.001)

### `pretty_print_policy(cols, rows, policy,goal)`
`pretty_print_policy()` is implemented as instructed. It takes the `cols` and `rows` of a world and given the resulting `policy` it maps its movements to actions. Those actions are then stored in a dictionary and later printed as concatenated strings. The position of the `goal` is used to identify and label it as 'G'.

In [15]:
from copy import deepcopy
def pretty_print_policy(cols, rows, policy,goal):
    action_map = {(0, -1):'^', (1, 0):'>', (0, 1):'v', (-1, 0):'<'}
    arrows = {state:action_map[policy[state]] for state in policy}   
    arrow_world = [[0 for i in range(cols)] for j in range(rows)]
    for k in arrows:
        if k[0]==goal[0] and k[1]==goal[1]:
            arrow_world[k[1]][k[0]] = 'G'            
        else:
            arrow_world[k[1]][k[0]] = arrows[k]
    for row in arrow_world:
        print(''.join(row))

In [16]:
cols = len(small_world[0])
rows = len(small_world)
pretty_print_policy(cols, rows, pi,(5,6))

v>>>>v
vv>>>v
vvv>>v
vvvv>v
vvvvvv
>>>>vv
>>>>>G


## Value Iteration

### Small World

In [17]:
small_world = read_world( "small.txt")

In [18]:
goal = (len(small_world[0])-1, len(small_world)-1)
gamma = 0.9

small_policy = value_iteration(small_world, costs, goal, 10, cardinal_moves, gamma, 0.7, 0.001)

In [19]:
cols = len(small_world[0])
rows = len(small_world)

pretty_print_policy(cols, rows, small_policy, goal)

v>>>>v
vv>>>v
vvv>>v
vvvv>v
vvvvvv
>>>>vv
>>>>>G


### Large World

In [20]:
large_world = read_world( "large.txt")

In [21]:
goal = (len(large_world[0])-1, len(large_world)-1) # Lower Right Corner FILL ME IN
gamma = 0.9

large_policy = value_iteration(large_world, costs, goal, 100000, cardinal_moves, gamma, 0.7, 0.00001)

In [22]:
cols = len(large_world[0])
rows = len(large_world)

pretty_print_policy( cols, rows, large_policy, goal)

v>>>>>>>>>>>>>>>>>>>>vvvvvv
vv>>>>>>>>>>>>>>vvvvvvvvvvv
vvvv>>>>>>>>>>>vvvvvvvvvvvv
vvvvv>>>>>>>>>>>vvvvvvvvvvv
vvvvv>>vv>>>>>>>>vvvvvvvvvv
vvvvvvvvv>>>>>>>>>vvvvvvvvv
vvvvvv>vvvvvvv>>>>vvvvvvvvv
vvvvv>vvv>>>vv>>vvvvvvvvvvv
vvv>>>vvv>>>>vvvvvvvvvvvvvv
vvv>>>vvvvvvvvvvvvvvvvvvvvv
vvv>>vvvvvvvvv>>>>vvvvvvvvv
vv>>>vvvvv>v>>>>>>vvvvvvvvv
vv>>>>vvvvvvvv>>>>>vvvvvvvv
vv>>>>vvvvvv>>>v>>>vvvvvvvv
vv>>>>>>vvvv>>>>>v>vvvvvvvv
vvv>v>>>vvvvvv>>>>v>vvvvvvv
vvv>>>>>>>vvvvvv>>vvvvvvvvv
vvv>>>>>>>>>vvvvvvvvvvvvvvv
vvvv>>>>>>>>>>>vvvvvvvvvvvv
vvvvv>>>>>>>>>>>>>vv>vvvvvv
vv>>>>>>>>>>>>>>>>>vvvvvvvv
v>>>>>>>>>>>>>>>>>>>>vvvvvv
>>>>>>>>>>>>>>>>>>>>>>>vvvv
>>>>>>>>>>>>>>>>>>>>>>>>vvv
>>>>>>>>>>>>>>>>>>>>>>>>vvv
>>>>>>>>>>>>>>>>>>>>>>>>>vv
>>>>>>>>>>>>>>>>>>>>>>>>>>G


## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.